# Province-Level Yield Statistics of China

## Data access
The data is provided by National Bureau of Statistics of China and can be freely accessed at https://data.stats.gov.cn

## Crops

- Rice (rice, rice_doublecropping_late, rice_early, rice_midseason_singlecropping_late)
- Wheat (wheat, spring_wheat, winter_wheat)
- Corn

In [1]:
selected_crop = ['RICE', 'RICE_DOUBLECROPPING_LATE', 'RICE_EARLY', 'RICE_MIDSEASON_SINGLECROPPING_LATE', 
                 'WHEAT', 'SPRING_WHEAT', 'WINTER_WHEAT', 'CORN']

## Export yield statistics for selected crops

- for year data, it is assummed to be harvest period and not planting.

In [54]:
import pandas as pd
import os

data_path = '/app/dev/AgML/china'

province_mapping = {'Shaanxi': 'CN-061',
 'Shanghai': 'CN-031',
 'Chongqing': 'CN-050',
 'Zhejiang': 'CN-033',
 'Jiangxi': 'CN-036',
 'Yunnan': 'CN-053',
 'Shandong': 'CN-037',
 'Liaoning': 'CN-021',
 'Tibet': 'CN-054',
 'Gansu': 'CN-062',
 'Qinghai': 'CN-063',
 'Beijing': 'CN-011',
 'Inner Mongolia': 'CN-015',
 'Hubei': 'CN-042',
 'Anhui': 'CN-034',
 'Guizhou': 'CN-052',
 'Ningxia': 'CN-064',
 'Jiangsu': 'CN-032',
 'Xinjiang': 'CN-065',
 'Shanxi': 'CN-014',
 'Hunan': 'CN-043',
 'Sichuan': 'CN-051',
 'Guangxi': 'CN-045',
 'Jilin': 'CN-022',
 'Hebei': 'CN-013',
 'Tianjin': 'CN-012',
 'Guangdong': 'CN-044',
 'Fujian': 'CN-035',
 'Heilongjiang': 'CN-023',
 'Henan': 'CN-041',
 'Hainan': 'CN-046'}

In [52]:

def restructure_df(data_path, province_mapping, selected_crop):
    
    list_df = []
    
    for crop in selected_crop:
        
        # load yield, production and area
        crop_yield_df = pd.read_csv(os.path.join(data_path, 'yield_{}.csv'.format(crop.lower())), 
                                    encoding = 'unicode_escape', engine ='python', skiprows=3)
        crop_area_df = pd.read_csv(os.path.join(data_path, 'sownarea_{}.csv'.format(crop.lower())), 
                                    encoding = 'unicode_escape', engine ='python', skiprows=3)
        crop_production_df = pd.read_csv(os.path.join(data_path, 'production_{}.csv'.format(crop.lower())), 
                                    encoding = 'unicode_escape', engine ='python', skiprows=3)
        
        # melt the dataframe
        crop_yield_df = pd.melt(crop_yield_df, id_vars=['Region'], var_name='year', value_name='yield')
        crop_area_df = pd.melt(crop_area_df, id_vars=['Region'], var_name='year', value_name='planted_area')
        crop_production_df = pd.melt(crop_production_df, id_vars=['Region'], var_name='year', value_name='production')

        # filter valid provinces values
        crop_yield_df = crop_yield_df[crop_yield_df['Region'].isin(list(province_mapping.keys()))]
        
        # join production and area to yield
        crop_yield_df = pd.merge(crop_yield_df, crop_area_df, on=['Region', 'year'], how='left')
        crop_yield_df = pd.merge(crop_yield_df, crop_production_df, on=['Region', 'year'], how='left')
        
        # assign crop name and country
        crop_yield_df['crop_name'] = crop
        crop_yield_df['country_code'] = 'CN'
        crop_yield_df =  crop_yield_df.rename(columns={'Region':'adm_id', 'year':'harvest_year'})
        crop_yield_df['adm_id'] = crop_yield_df['adm_id'].map(province_mapping)
        list_df.append(crop_yield_df)
        

    # concatenate df for several crops
    master_df = pd.concat(list_df, axis=0)
    
    # remove nan in yield and values less than 0
    master_df = master_df.dropna(subset=['yield'])
    master_df = master_df[master_df['yield'] > 0]
    
    # conform to agml standard
    master_df['season_name'] = 'N/A'
    master_df['planting_year'] = 'N/A'
    master_df['planting_date'] = 'N/A'
    master_df['harvest_date'] = 'N/A'
    
    # scale production and area according to multiply value provided with data
    master_df['planted_area'] = master_df['planted_area'] * 1000
    master_df['production'] = master_df['production'] * 10000

    # convert yield from kg/ha to mt/ha
    master_df['yield'] = master_df['yield'] / 1000

    # assign data source
    # master_df['source'] = 'https://data.stats.gov.cn'   

    # reorder to conform to agml standard
    master_df = master_df.reindex(columns=['crop_name', 'country_code', 'adm_id', 'season_name', 
                                          'planting_year', 'planting_date', 'harvest_year', 'harvest_date',\
                                          'yield', 'production', 'planted_area', 'harvest_area'])
    
    return master_df

In [57]:
master_df = restructure_df(data_path, province_mapping, selected_crop)
master_df.head(10)

,crop_name,country_code,adm_id,season_name,planting_year,planting_date,harvest_year,harvest_date,yield,production,planted_area,harvest_area
31,RICE,CN,CN-011,N/A,N/A,N/A,2022,N/A,5.06215,2100.0,420.0,NaN
32,RICE,CN,CN-012,N/A,N/A,N/A,2022,N/A,9.52293,526200.0,55260.0,NaN
33,RICE,CN,CN-013,N/A,N/A,N/A,2022,N/A,6.38113,488800.0,76590.0,NaN
34,RICE,CN,CN-014,N/A,N/A,N/A,2022,N/A,6.75000,14600.0,2160.0,NaN
35,RICE,CN,CN-015,N/A,N/A,N/A,2022,N/A,7.69526,902100.0,117230.0,NaN
36,RICE,CN,CN-021,N/A,N/A,N/A,2022,N/A,8.24102,4255600.0,516390.0,NaN
37,RICE,CN,CN-022,N/A,N/A,N/A,2022,N/A,8.17245,6809100.0,833180.0,NaN
38,RICE,CN,CN-023,N/A,N/A,N/A,2022,N/A,7.54714,27180000.0,3601370.0,NaN
39,RICE,CN,CN-031,N/A,N/A,N/A,2022,N/A,7.97778,827300.0,103700.0,NaN
40,RICE,CN,CN-032,N/A,N/A,N/A,2022,N/A,8.96550,19916100.0,2221420.0,NaN


## Save the data

In [161]:
# master_df.to_csv('/app/dev/AgML/' + "YIELD_CHINA.csv")

## Summary by crop

In [7]:
summary = {'CROP':[],
           'MIN_YEAR':[],
           'MAX_YEAR':[],
           'NUM_YEARS':[],
           'NUM_DISTRICTS':[],
           'DATA_SIZE':[],
           'NAN_IN_YIELD':[],
           'LESS_THAN_ZERO_YIELD':[]
          }

for crop in selected_crop:
    
    crop_yield_df = master_df[master_df['crop_name'] == crop]
    list_years = sorted(crop_yield_df["harvest_year"].unique())
    data_size = crop_yield_df["yield"].count()
    nan_counts = crop_yield_df['yield'].isnull().sum()
    district_counts = len(crop_yield_df['adm_id'].unique())
    less_than_zero = len(crop_yield_df[crop_yield_df['yield'] <= 0 ])
    
    summary['CROP'].append(crop)
    summary['MIN_YEAR'].append(min(list_years))
    summary['MAX_YEAR'].append(max(list_years))
    summary['NUM_YEARS'].append(len(list_years))
    summary['NUM_DISTRICTS'].append(district_counts)
    summary['DATA_SIZE'].append(crop_yield_df.shape[0])
    summary['NAN_IN_YIELD'].append(nan_counts)
    summary['LESS_THAN_ZERO_YIELD'].append(less_than_zero)

district_summary = pd.DataFrame(summary)
district_summary = district_summary.sort_values(by='CROP')
district_summary.head(25)

,CROP,MIN_YEAR,MAX_YEAR,NUM_YEARS,NUM_DISTRICTS,DATA_SIZE,NAN_IN_YIELD,LESS_THAN_ZERO_YIELD
7,CORN,1990,2022,33,31,998,0,0
0,RICE,1990,2022,33,30,983,0,0
1,RICE_DOUBLECROPPING_LATE,1990,2022,33,15,405,0,0
2,RICE_EARLY,1990,2022,33,15,382,0,0
3,RICE_MIDSEASON_SINGLECROPPING_LATE,1990,2022,33,30,924,0,0
5,SPRING_WHEAT,1990,2018,28,21,407,0,0
4,WHEAT,1990,2022,33,30,981,0,0
6,WINTER_WHEAT,1990,2022,33,30,821,0,0
